# Projet : Détection de la Dépression chez les Étudiants

**Objectif** : Prédire si une personne est en dépression à partir de données cliniques et socio-professionnelles.

---

## Métriques de performance

Les classes sont **fortement déséquilibrées** : 18% de cas de dépression, 82% de cas non-dépressifs.

- **Recall (sensibilité)** : TP / (TP + FN) → éviter de **manquer un cas positif**  
- **Precision (précision)** : TP / (TP + FP) → limiter les **faux positifs**  
- **F1-score** → compromis entre précision et recall

**Objectif ciblé :** F1 ≈ 0.5 et Recall ≈ 0.7

## 1. Imports et Configuration

In [ ]:
import sys
import os

# Ajouter le dossier parent au path pour importer src
sys.path.insert(0, os.path.dirname(os.getcwd()))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, recall_score

# Modules personnalisés
from src.preprocessing import set_trainset, preprocessing, test_preprocessing, TARGET_FEATURE
from src.modeling import create_models, get_adaboost_hyperparams, optimize_model, get_best_model, create_preprocessor
from src.evaluation import evaluation, evaluate_multiple_models, plot_precision_recall_curve, model_final, evaluate_with_threshold

import warnings
warnings.filterwarnings("ignore")

# Configuration
TRAIN_PATH = '../data/train.csv'
TEST_PATH = '../data/test.csv'
SUBMISSION_PATH = '../data/submission.csv'
RANDOM_STATE = 42
TEST_SIZE = 0.2
CV_FOLDS = 4
OPTIMAL_THRESHOLD = -0.01

print("Modules chargés avec succès !")

## 2. Chargement des Données

In [ ]:
# Charger les données
data = pd.read_csv(TRAIN_PATH)
df = data.copy()

print(f"Shape: {df.shape}")
df.head()

## 3. Résumé de l'EDA

### Principaux insights :

- **Classes déséquilibrées** : 18% dépressifs / 82% non-dépressifs  
- **Étudiants plus touchés** que les travailleurs  
- **Tranche 20-30 ans** particulièrement affectée  
- **Pression** académique et professionnelle plus élevée chez les déprimés  
- **Hygiène de vie** et Financial Stress semblent influencer la dépression  
- **Lien fort** entre idées suicidaires et dépression  
- Certaines variables ont >80% de NaN → traitement spécifique

In [ ]:
# Visualisation de la target
print(f"Répartition de {TARGET_FEATURE}:")
print(df[TARGET_FEATURE].value_counts(normalize=True))

plt.figure(figsize=(8, 5))
df[TARGET_FEATURE].value_counts().plot(kind='bar', color=['steelblue', 'coral'])
plt.title('Distribution de la variable cible (Depression)')
plt.xlabel('Depression')
plt.ylabel('Count')
plt.xticks([0, 1], ['Non (0)', 'Oui (1)'], rotation=0)
plt.tight_layout()
plt.show()

## 4. Pre-processing

### Stratégie :
1. **Train/Test Split** (80/20)
2. **Encodage** des variables catégorielles
3. **Imputation** des valeurs manquantes
4. **Combinaison** des colonnes similaires (Pressure, Satisfaction)

--> Dataset réduit à **22353 rows × 16 columns** pour X_train

In [ ]:
# Split Train/Test
trainset, testset = train_test_split(df, test_size=TEST_SIZE, random_state=RANDOM_STATE)

print(f"Trainset shape: {trainset.shape}")
print(f"Testset shape: {testset.shape}")

print(f"\nProportions de {TARGET_FEATURE} dans le trainset:")
print(trainset[TARGET_FEATURE].value_counts())

In [ ]:
# Définir le trainset global pour l'encodage
set_trainset(trainset)

# Preprocessing
X_train, y_train = preprocessing(trainset.copy())
X_test, y_test = preprocessing(testset.copy())

print(f"\nX_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

In [ ]:
# Aperçu des données preprocessées
X_train.head()

## 5. Modélisation

### Modèles évalués :
- **RandomForest**
- **AdaBoost**
- **SVM**
- **KNN**

In [ ]:
# Création des modèles
models = create_models()
print("Modèles créés:", list(models.keys()))

In [ ]:
# Évaluation de tous les modèles
trained_models = evaluate_multiple_models(models, X_train, y_train, X_test, y_test)

## 6. Optimisation avec GridSearchCV

Basé sur les résultats précédents, **AdaBoost** offre le meilleur compromis.

Optimisation des hyperparamètres :
- `n_estimators`
- `learning_rate`
- `algorithm`
- `estimator` (DecisionTree avec différents max_depth)

In [ ]:
# GridSearchCV sur AdaBoost
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import AdaBoostClassifier

preprocessor = create_preprocessor()
AdaBoost = make_pipeline(preprocessor, AdaBoostClassifier(random_state=RANDOM_STATE))

hyper_params = get_adaboost_hyperparams()
print("Hyperparamètres à tester:")
for k, v in hyper_params.items():
    print(f"  {k}: {v}")

In [ ]:
# Optimisation
grid = optimize_model(AdaBoost, X_train, y_train, hyper_params, scoring='recall', cv=CV_FOLDS)

In [ ]:
# Évaluation du meilleur modèle
y_pred = grid.predict(X_test)
print("\nRapport de classification du meilleur modèle:")
print(classification_report(y_test, y_pred))

In [ ]:
# Courbes d'apprentissage du meilleur modèle
_ = evaluation(grid.best_estimator_, X_train, y_train, X_test, y_test)

## 7. Courbe Precision-Recall

Permet de trouver le seuil optimal selon nos priorités :
- **Priorité au Recall** : éviter de manquer des cas de dépression

In [ ]:
# Courbe Precision-Recall
precision, recall, threshold = plot_precision_recall_curve(grid.best_estimator_, X_test, y_test)

In [ ]:
# Évaluation avec seuil personnalisé
print(f"Évaluation avec threshold = {OPTIMAL_THRESHOLD}:")
metrics = evaluate_with_threshold(grid.best_estimator_, X_test, y_test, threshold=OPTIMAL_THRESHOLD)

## 8. Bilan

### Objectif initial :
- F1 ≥ 0.5
- Recall ≥ 0.7

### Résultats obtenus :
- **F1 Score : ~0.86**
- **Recall : ~0.94**

**Objectif largement dépassé !**

## 9. Submission (Kaggle)

In [ ]:
# Entraînement du modèle final
final_model = get_best_model()
final_model.fit(X_train, y_train)

print("Modèle final entraîné !")

In [ ]:
# Charger les données de test Kaggle
df_test = pd.read_csv(TEST_PATH)
test_ids = df_test['id'].values

print(f"Test data shape: {df_test.shape}")
df_test.head()

In [ ]:
# Preprocessing des données de test
df_test_processed = test_preprocessing(df_test.copy())

print(f"Test data preprocessed shape: {df_test_processed.shape}")

In [ ]:
# Prédictions avec seuil
predictions = model_final(final_model, df_test_processed, threshold=OPTIMAL_THRESHOLD)
predictions = predictions.astype(int)

print(f"Predictions shape: {predictions.shape}")
print(f"Distribution: {np.bincount(predictions)}")

In [ ]:
# Créer le fichier de submission
submission = pd.DataFrame({
    'id': test_ids,
    'Depression': predictions
})

submission.to_csv(SUBMISSION_PATH, index=False)
print(f"Fichier submission.csv créé !")

submission.head()